In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from User_Classes import UserClass
from Advertising_Environment import calculate_margin, calculate_price_index
from Clairvoyant import find_optimal_bid_for_class
from Advertising.GPTS_Learner import GPTS_Learner
from Advertising.GPUCB_Learner import GPUCB_Learner

import warnings
warnings.filterwarnings("ignore")

# Environment

In [ ]:
#Environment
n_arms = 100

prices = [50, 100, 150, 200, 250]
bids = np.linspace(0.01, 3.0, n_arms)
sigma = 200

user = UserClass(name='C1')

T = 365
n_experiments = 5

gpts_rewards_per_experiment = []
gpucb_rewards_per_experiment = []


optimum_bid = find_optimal_bid_for_class(user.user_index, calculate_price_index(price))

In [ ]:
class Pricing_Environment():
    def __init__(self, n_arms, probabilities, prices=None):
        self.n_arms = n_arms
        self.probabilities = probabilities
        self.prices = prices

    def round(self, pulled_arm, user_c, n_trials=1):
        successes = np.random.binomial(n_trials, self.probabilities[user_c.index][pulled_arm])
        number_returns = np.random.poisson(successes * self.poissons[user_c.index])
        marg = self.margins[pulled_arm] * (number_returns + successes)
        return successes, number_returns

In [ ]:
def fun(user_class, bid, price):
    mean_per_bid = user_class.get_click_bids(bid) * (user_class.get_conversion_per_price(price) * calculate_margin(price)) - user_class.get_click_bids(bid) * user_class.get_cost_per_click(bid)    #sample_cost
    return mean_per_bid

class BiddingEnvironment:
    def __init__(self, bids, sigma, user_class, price, n_arms):
        self.bids = bids
        self.means = self.initialize_means(user_class=user_class, bids=bids, price=price)
        self.sigmas = np.ones(len(bids)) * sigma
        self.price = price
        self.n_arms = n_arms

    def initialize_means(self, user_class, bids, price):
        means = np.zeros(len(bids))
        for i in range(len(means)):
                means[i] = fun(user_class, bids[i], price)
        return means

    def round(self, pulled_arm):
        return np.random.normal(self.means[pulled_arm], self.sigmas[pulled_arm])